In [1]:
print("hi")
from transformers import TFBertModel, BertTokenizer,BertConfig,BertModel
import re
import numpy as np
import torch

hi


/home/pavit21178/anaconda3/envs/conda_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# Load the ProtBert tokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)

# Load the ProtBert model
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

# Set model to evaluation mode

import pandas as pd
df=pd.read_csv('merged_data_modified.csv')
y=df['id']
x=df['seq']

#split the data into train and test






In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#oversample the data to create a balanced dataset in train
#use random oversampling
print(x_train.shape)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train=x_train.values.reshape(-1,1)
x_train, y_train = ros.fit_resample(x_train, y_train)
x_train=x_train.reshape(-1)





(353,)


In [4]:
print(x_train.shape,y_train.shape)

(494,) (494,)


In [5]:
x_train=[re.sub(r"[UZOB]", "X", seq) for seq in x_train]
x_test=[re.sub(r"[UZOB]", "X", seq) for seq in x_test]

In [6]:
#make a space seaprated string of the sequences
for i in range(len(x_train)):
    starting_string=x_train[i]
    inserting_string=''
    for j in range(len(starting_string)):
        inserting_string+=starting_string[j]+' '
    inserting_string=inserting_string[:-1]
    x_train[i]=inserting_string

#do the same for the test data
for i in range(len(x_test)):
    starting_string=x_test[i]
    inserting_string=''
    for j in range(len(starting_string)):
        inserting_string+=starting_string[j]+' '
    #remove the last space
    inserting_string=inserting_string[:-1]
    x_test[i]=inserting_string

In [7]:
print(x_train[0])

Y G G T P P F V


In [8]:
#use gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:


ids = tokenizer.batch_encode_plus(x_train,max_length=40,pad_to_max_length=True, return_tensors="pt")
hidden_states = model(**ids)[0]


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/pavit21178/anaconda3/envs/conda_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
#get sequence layer
sequence_output = hidden_states[0]
print(sequence_output.shape)


torch.Size([40, 1024])


In [11]:
print(hidden_states.shape)
#what is the data type of hidden_states
print(type(hidden_states))

torch.Size([494, 40, 1024])
<class 'torch.Tensor'>


In [12]:
#do same with test data
ids = tokenizer.batch_encode_plus(x_test,max_length=40,pad_to_max_length=True, return_tensors="pt")
test_hidden_states = model(**ids)[0]
print(test_hidden_states.shape)

/home/pavit21178/anaconda3/envs/conda_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([89, 40, 1024])


In [13]:
print(x_train)

['Y G G T P P F V', 'Q L', 'G P G', 'P F P', 'H C H T N', 'K G S L A D E E', 'T A Y', 'F F R P F F R P F F', 'G G N P', 'A D E', 'P F P G P I', 'G G L G', 'R P F F R P F F', 'P P', 'Q L F G P N V N P W H N P', 'P F P G P I P', 'D G', 'D E S', 'Y Q E P V L G P V R G P F P I', 'V Y P', 'D E D', 'R P G', 'E E E Q', 'A P K H K E M P F P K Y P V E P F', 'G I', 'E G G', 'S A E Q K', 'E N', 'V G', 'V D', 'N E N L L', 'F G G F', 'E D V', 'E D', 'R G P F I V', 'F Y P E L F R', 'D P Q', 'R R P P', 'G G R P F F G G', 'G L', 'P V A R M C R', 'E E L', 'V Y P F G G G I N H', 'D D D D', 'Y G Y', 'L E Q L L', 'K P F', 'G R P', 'P A Q', 'P F P G P I P N S', 'R G P F P I V', 'I S', 'Y P', 'L G L', 'F F G G', 'G G P', 'K F', 'Y F', 'N L Q G', 'Q L F N P S T N P W H S P', 'G E', 'Y Q Q P V L G P V R G P F P I', 'R G P P F', 'L R L', 'G G R P F F', 'P G R', 'R G P P G G F F', 'N Y', 'M P F P K Y P V E P F', 'G P F P I I V', 'N G K E T', 'G F F', 'P G P G P G', 'P F', 'G G L G G', 'F F P P', 'D G G', 'E F',

In [14]:
print(hidden_states[0])

tensor([[ 0.0125,  0.0013,  0.1073,  ..., -0.1376, -0.0970, -0.0342],
        [ 0.0745,  0.0300, -0.0185,  ...,  0.1138, -0.0163,  0.0793],
        [-0.0249,  0.0129,  0.1416,  ..., -0.0964, -0.1221,  0.0566],
        ...,
        [ 0.0582,  0.0498,  0.0815,  ..., -0.0452, -0.0270,  0.0789],
        [ 0.0590,  0.0885,  0.0679,  ..., -0.0253,  0.0055,  0.0735],
        [ 0.0869,  0.0188, -0.0205,  ..., -0.0127, -0.0330,  0.0567]],
       grad_fn=<SelectBackward0>)


In [15]:
print(test_hidden_states.shape)


torch.Size([89, 40, 1024])


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader



In [17]:
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(40, 16, kernel_size=3, padding=1)  
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * (1024 // 4), 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = x.view(-1, 40, 1024)  # Reshape to (batch_size, channels, width)
        x = nn.functional.relu(self.conv1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 32 * (1024 // 4))
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [18]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        train_loss = 0.0
        train_acc = 0.0
        for batch in train_loader:
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            optimizer.zero_grad(set_to_none=True)
            print(inputs.shape)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward(retain_graph=True)
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)  # Get the predicted class indices
            train_acc += torch.sum(preds == labels.data).item()
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_acc / len(train_loader.dataset)
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        print(f'Epoch: {epoch}')
        print(f'\tTraining Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}')
        print(f'\tValidation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

def validate(model, data_loader, criterion, device):
    val_loss = 0.0
    val_acc = 0.0
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch[0].to(device)
            outputs = model(inputs)
            labels = batch[1].to(device)  # Get the actual labels from the data loader
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)  # Get the predicted class indices
            val_acc += torch.sum(preds == labels.data).item()
        val_loss = val_loss / len(data_loader.dataset)
        val_acc = val_acc / len(data_loader.dataset)
    return val_loss, val_acc

In [19]:
batch_size = 32
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

dataset = CustomDataset(hidden_states, y_train)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset = CustomDataset(test_hidden_states, y_test)
val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


In [20]:
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Train the model
num_epochs = 10
train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

cuda
torch.Size([32, 40, 1024])


KeyboardInterrupt: 

In [ ]:
#run the model
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
train(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

# Convert the hidden states to a PyTorch tensor
y_train = torch.tensor(y_train)

# Train the model
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    output = model(hidden_states)
    loss = criterion(output, y_train)
    loss.backward(retain_graph=True)
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    print("done")

In [ ]:
# Evaluate the model
model.eval()
y_test = torch.tensor(y_test.values)
with torch.no_grad():
    test_hidden_states = test_hidden_states.detach()
    output = model(test_hidden_states)
    print(output)
    print(output.shape)
    print(y_test)
    print(y_test.shape)
    loss = criterion(output, y_test)
    print(f"Test Loss: {loss.item()}")
    print("done")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (89x9690 and 6630x2)

In [ ]:
#print the accuracy
_, predicted = torch.max(output, 1)
correct = (predicted == y_test).sum().item()
accuracy = correct / y_test.size(0)

In [ ]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.7865168539325843


In [ ]:
#run the model
model = build_CNN_classifier_model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(hidden_states, y_train, epochs=10, batch_size=32, validation_data=(test_hidden_states, y_test))



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 40, 1024)]        0         
                                                                 
 conv1d_4 (Conv1D)           (None, 39, 128)           262272    
                                                                 
 conv1d_5 (Conv1D)           (None, 39, 64)            8256      
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                           

TypeError: Cannot convert the argument `type_value`: torch.float32 to a TensorFlow DType.

In [ ]:
bitter_classifier = build_CNN_classifier_model()
output_test = bitter_classifier(test_outputs) # tensor [batch_size, max_length, 1024]
output_test

ValueError: Missing data for input "input_2". You passed a data dictionary with keys ['last_hidden_state', 'pooler_output']. Expected the following keys: ['input_2']